In [12]:

import numpy as np
import matplotlib.pyplot as plt
import pdb

# データセットの用意
検証には引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
# データの前処理
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [4]:
# One-Hot-encoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_o = enc.fit_transform(y_train[:, np.newaxis])
y_test_o = enc.fit_transform(y_test[:, np.newaxis])
print(y_train.shape)
print(y_train_o.shape)

(60000,)
(60000, 10)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_o, test_size=0.2)
print(X_train.shape)
print(X_val.shape)

(48000, 784)
(12000, 784)


# 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。


ここでは パディング は考えず、ストライド も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。

In [6]:
def forward(self, x, w, b):
    """
    パラメータ
    ---------------------
    x : 入力値

    return
    ---------------------
    a　：　出力
    """
    # filter_size
    filter_size = w.shape[0]

    # z の要素数 
    x_shape = x.shape[0]

    # aのから配列
    a = np.zeros(x_shape - (filter_size -1))
    for i in range(len(a)):
        a[i] = (x[i : i + filter_size] @ w) + b
    return a.astype(int)

In [7]:
def update_based(self, layer):
    layer.w -= layer.lr * layer.dw
    layer.b -= layerlr * layer.db
    return layer

In [8]:
def backward(x, w, b, da):
    """
    パラメータ
    ---------------------
    delta_a : 入力値
    
    return
    ---------------------
    z　：　出力
    """    
    filter_size = x.shape[0] -1
    x_size = x.shape[0]
    w_size = w.shape[0]
    
    # w, bの勾配計算
    dw = np.zeros(filter_size)
    
    for i in range(filter_size):
        dw[i] = x[i : i + da.shape[0]] @ da
    db = da.sum(axis=0)
    
    dx = np.zeros(x_size)
    for j in range(x_size):
        back_filter = 0
        for s in range(w_size):
            if j-s < 0 or j-s > len(n_out) - 1:
                back_filter += 0 * w[s]
            else:
                back_filter += da[j - s] * w[s]
        dx[j] = back_filter   
        
    return dx, dw, db
    

### チャンネル数を1に限定した1次元畳み込み層クラス

In [9]:
class SimpleConv1d():
    """
      １次元畳み込み層のクラス
    """
    
    def forward(self, x, w, b):
        """
        パラメータ
        ---------------------
        x : 入力値

        return
        ---------------------
        a　：　出力
        """
        self.x = x
        self.w = w
        self.b = b
        
        # filter_size
        self.filter_size = w.shape[0]
        # x の要素数 
        self.x_size = x.shape[0]
        # wの要素数
        self.w_size = w.shape[0]

        # aのから配列
        self.a = np.zeros(self.x_size - (self.filter_size -1))
        for i in range(len(self.a)):
            self.a[i] = (self.x[i : i + self.filter_size] @ self.w) + self.b
        return self.a.astype(int)
    
    def backward(self, da):
        """
        パラメータ
        ---------------------
        delta_a : 入力値

        return
        ---------------------
        z　：　出力
        """ 

        # w, bの勾配計算
        dw = np.zeros(self.filter_size -1)

        for i in range(self.filter_size -1):
            dw[i] = self.x[i : i + da.shape[0]] @ da
        db = da.sum(axis=0)

        dx = np.zeros(self.x_size)
        for j in range(len(dx)):
            lox = 0
            for s in range(self.w_size):
                if j-s < 0 or j-s > len(self.a) - 1:
                    lox += 0 * self.w[s]
                else:
                    lox += da[j - s] * self.w[s]
            dx[j] = lox  
            
        return dx, dw, db
    
    def out_size(n_in, p, f, s):
        """
        n_in :  入力サイズ 
        p : ある方向へのパディング数
        f :  フィルタのサイズ
        s : ストライドのサイズ
        """
        out = (n_in + (2*p) - f) / s + 1

        return out

# 【問題2】1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

In [10]:
def out_size(n_in, p, f, s):
    """
    n_in :  入力サイズ 
    p : ある方向へのパディング数
    f :  フィルタのサイズ
    s : ストライドのサイズ
    """
    n_out = (n_in + (2*p) - f) / s + 1
    
    return n_out

# 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。


入力x、重みw、バイアスbを次のようにします。

In [11]:
#　入力
x = np.array([1,2,3,4])
# 重み
w = np.array([3, 5, 7])
# バイアス
b = np.array([1])
x.shape

(4,)

In [12]:
sc1 = SimpleConv1d()
n_out = sc1.forward(x, w, b)
n_out

array([35, 50])

In [13]:
# バックプロパゲーションの誤差
da = np.array([10, 20])

In [14]:
dx, dw, db = sc1.backward(da)
print(dx)
print(dw)
print(db)

[ 30. 110. 170. 140.]
[50. 80.]
30


# 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。

In [15]:
x_ = np.array([[1,2,3,4],
               [2,3,4,5]])


w_ = np.array([[[1,1,2],[2,1,1]],
              [[2,1,1],[1,1,1]],
              [[1,1,1],[1,1,1]]])


b_ = np.array([1,2,3])

# フォワードの出力
out_ = np.array([[21,29],
                [18,25],
                [18,24]])

b_.reshape(-1, 1)

array([[1],
       [2],
       [3]])

In [16]:
x_.reshape(2, 1, 4)

array([[[1, 2, 3, 4]],

       [[2, 3, 4, 5]]])

In [17]:
def forward(x, w, b):
    """
    パラメータ
    ---------------------
    x : 入力値
    
    return
    ---------------------
    a　：　出力
    """
    #出力チャネル、入力チャネル、filter数
    FN, C, FS = w.shape # (出力チャンネル数、入力チャンネル数、フィルタサイズ)
    C, W = x.shape # (入力チャンネル数、特徴量数)

    # 出力するaの空配列を作成
    a = np.zeros((FN, (W - (FS - 1)))) # (出力チャンネル数、特徴量数)
    # aを計算して出力
    for i in range(FN):
        index = np.empty((0, FS), int)
        for j in range(W - (FS - 1)):
            index_to_add = np.arange(j, j+FS)
            index = np.append(index, [index_to_add], axis=0)
        a[i] = (x[:, index] * w[i]).sum(axis=1).sum(axis=1)
    a += b.reshape(-1,1)
    return a  

In [18]:
n_out = forward(x_, w_, b_)
n_out

array([[21., 29.],
       [18., 25.],
       [18., 24.]])

In [19]:
#delta_aの値
loss_ = np.array([[9,11],
                [32,35],
                [52,56]])

# バックワードの勾配
x_delta = np.array([[125,230,204,113],
                    [102,206,195,102]])


w_delta = np.array([[[31,51,71],[51,71,91]],
                    [[102,169,236],[169,236,303]],
                    [[164,272,380],[272,380,488]]])
loss_.shape

(3, 2)

In [20]:
def cv_backward(x_, w_, loss_):
    """
    パラメータ
    ---------------------
    delta_a : 入力値
    
    return
    ---------------------
    z　：　出力
    """
    #１行のx要素数
    x_size = x_[0].shape[0]
    
    x_size2 = x_[0].shape[0] -1
    # xのチャネル数
    x_cha = x_.shape[0]
    # wの要素数
    w_size = w_[0].shape[1]
    # loss要素数
    loss_size = loss_[0].shape[0]
    # lossのチャネル数
    loss_cha = loss_.shape[0]
    #１行のn_out要素数
    n_out_size = n_out.shape[1]

    # w, bの勾配計算
    dw = np.zeros([loss_cha,x_cha, x_size2])
    for k in range(loss_cha):
        #各チャネル保管
        f_2 = np.zeros([loss_size,  x_cha, x_size2])
        for j in range(loss_size):
            #conv一時保管
            f_1 = np.zeros([x_cha, x_size2])
            for i in range(x_cha):
                f_1[i] = x_[j][i : i + x_size2] * loss_[k][j]
            f_2[j] = f_1
        dw[k] = np.sum(f_2, axis=0)
    
    # bの勾配計算
    db = loss_.sum(axis=1)
    
    #デルタxの計算
    dx = np.zeros([x_cha, x_size])
    b_1 = np.zeros([x_cha, x_size])
    b_2 = np.zeros(x_size)
    for l in range(loss_cha):
        for k in range(x_cha):
            for j in range(x_size):
                back_filter = 0
                for s in range(w_size):
                    if j-s < 0 or j-s > n_out_size - 1:
                        back_filter += 0 * w_[l][k][s]
                    else:
                        back_filter += loss_[l][j - s] * w_[l][k][s]
                b_2[j] = back_filter
            b_1[k] = b_2
        dx += b_1
        
    return dx, dw, db

In [21]:
dx, dw, db = cv_backward(x_, w_, loss_)

In [22]:
dx

array([[125., 230., 204., 113.],
       [102., 206., 195., 102.]])

In [23]:
dw

array([[[ 31.,  51.,  71.],
        [ 51.,  71.,  91.]],

       [[102., 169., 236.],
        [169., 236., 303.]],

       [[164., 272., 380.],
        [272., 380., 488.]]])

In [24]:
db

array([ 20,  67, 108])

In [25]:
class Conv1d:
    """
    チャネル数を限定しない１次元畳み込み層のクラス
    ----------------------------
    parameter
    ----------------------------
    FN: 出力チャネル
    FC：入力チャネル
    FS：　フィルター要素数
    """
    def __init__(self, FN, C, FS, sigma=0.01):
        self.sigma = sigma
        self.w = self.sigma * np.random.randn(FN, C, FS)
        self.b = np.zeros((FN, 1))
        self.h_w = self.sigma * np.random.randn(FN, C, FS)
        self.h_b = np.zeros((FN, 1))

    def forward(self, x):
        """
        パラメータ
        ---------------------
        x : 入力値

        return
        ---------------------
        a　：　出力
        """
        self.x = x
        #出力チャネル、入力チャネル、filter数
        FN, FC,FS  = self.w.shape
        C, W = self.x.shape

        # 出力するaの空配列を作成
        self.n_out = np.zeros((FN, (W - (FS - 1)))) # (出力チャンネル数、特徴量数)
        # aを計算して出力
        for i in range(FN):
            index = np.empty((0, FS), int)
            for j in range(W - (FS - 1)):
                index_to_add = np.arange(j, j+FS)
                index = np.append(index, [index_to_add], axis=0)
            self.n_out[i] = (x[:, index] * self.w[i]).sum(axis=1).sum(axis=1)
        self.n_out += self.b.reshape(-1,1)
        return self.n_out  
    
    def backward(self, loss):
        """
        パラメータ
        ---------------------
        delta_a : 入力値

        return
        ---------------------
        z　：　出力
        """
        #出力チャネル、入力チャネル、filter数
        FN, FC,FS  = self.w.shape
        C, W = self.x.shape
        LC, LS = loss.shape 
        print(self.n_out.shape)
        n_out_size = self.n_out.shape[1]
        #１行のx要素数
        self.dw = np.zeros(self.w.shape)
        
           # w, bの勾配計算
        dw = np.zeros([FN, FC, FS])
        for k in range(LC):
            #各チャネル保管
            f_2 = np.zeros([FN, FC, FS])
            for j in range(LS):
                #conv一時保管
                f_1 = np.zeros([C, FS])
                for i in range(C):
                    for s in range(W - (FS-1)):
                        f_1[i] = self.x[i][s : s + FS] * loss_[k][j]
                f_2[j] = f_1
            dw[k] = np.sum(f_2, axis=0)

        # bの勾配計算
        self.db = loss_.sum(axis=1).reshape(-1, 1)

        #デルタxの計算
        #デルタxの計算
        dx = np.zeros([C, W])
        b_1 = np.zeros([C, W])
        b_2 = np.zeros(W)
        for l in range(LC):
            for k in range(C):
                for j in range(W):
                    back_filter = 0
                    for s in range(FS):
                        if j-s < 0 or j-s > n_out_size - 1:
                            back_filter += 0 * self.w[l][k][s]
                        else:
                            back_filter += loss[l][j - s] * self.w[l][k][s]
                    b_2[j] = back_filter
                b_1[k] = b_2
            dx += b_1
            self = self.update_based(self)
            return dx
    
    def update_based(self, layer):
        layer.w -= 0.001 * layer.dw
        layer.b -= 0.001 * layer.db
        return layer

In [26]:
CV1 = Conv1d(FN=3,  C=2, FS=3)

In [27]:
CV1.forward(x_)

array([[ 0.08589167,  0.11366768],
       [-0.07352675, -0.09218305],
       [ 0.01574792,  0.02128156]])

In [28]:
CV1.backward(loss_)

(3, 2)


array([[-0.01268228,  0.07519487,  0.03841887, -0.08852691],
       [ 0.03331255,  0.13008874,  0.23095026,  0.14876413]])

# 【問題5】（アドバンス課題）パディングの実装
畳み込み層にパディングの機能を加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。


最も単純なパディングは全て0で埋める ゼロパディング であり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。


フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。なお、NumPyにはパディングの関数が存在します。



# 【問題6】（アドバンス課題）ミニバッチへの対応
ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。

# 【問題7】（アドバンス課題）任意のストライド数
ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください。

# 【問題8】学習と推定
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。


出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 平滑化 を行なってください。


画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [150]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.w = initializer.W( self.n_nodes1, self.n_nodes2)
        self.b =  initializer.B(self.n_nodes2)
        # adagrad用
        #if Adagrad = True: 
        self.h_w = np.zeros((n_nodes1, n_nodes2))
        self.h_b = np.zeros(n_nodes2)

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """   
        # 前層が畳み込み層の場合、出力チャンネル数に該当する次元を減らす
        if len(X.shape) > 2:
            self.X_shape = X.shape # backward時の復元用にshapeは保存しておく
            X = X.reshape(X.shape[1], X.shape[0] * X.shape[2])
        self.X = X
        
        self.A =  np.dot(X, self.w) + self.b
        return self.A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.db = np.sum(dA, axis=0)
#         print(self.X.shape)
#         print(dA.shape)
        self.dw = np.dot(self.X.T, dA)
#         print(dA.shape)
#         print(self.w.T.shape)
        dZ = np.dot(dA, self.w.T)
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [151]:
class Softmax():
    def forward(self, A):
        # overflow対策
        A_max = np.max(A, axis=1)
        exp_A = np.exp(A - A_max.reshape(-1, 1))
        sum_exp_A = np.sum(exp_A, axis=1).reshape(-1, 1)
        return exp_A / sum_exp_A
    
    def backward(self, Z, Y):
        loss = self._cross_entropy(Z, Y)
        D  = Z - Y
        return D, loss 
    
    def _cross_entropy(self,Z, y):
        """
        パラメータ
        ーーーーーーーーーーーーー
        y : 正解ラベル（one-hot表現）
        z３　：　クラスの確率
        n : バッチサイズ

        """
        if y.ndim == 1:    # 次元が 1 の場合
            y = y.reshape(1, y.size)
        batch_size = y.shape[0]
        return -1* np.sum(y * np.log(Z)) / batch_size

In [152]:
class Relu():
    def forward(self, A):
        self.mask = (A <= 0)
        a = A.copy()
        self.A = A
        a[self.mask] = 0
        return np.maximum(0, A)
      
    def backward(self, dA):
        #dA[self.mask] = 0
        out =  np.where(self.A > 0., 1., 0.)
        return out
 

In [153]:
class XavierInitializer():
    """
    Xavierによる初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
      
    """
    def __init__(self, sigma):
        self.sigma = 1/ np.sqrt(sigma)
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :　重みの初期値
        """
        self.w = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return self.w
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :　バイアスの初期値
        """
        self.b = self.sigma * np.random.randn(n_nodes2)
        return self.b

In [154]:
class HeInitializer():
    
    def __init__(self, sigma):
        self.sigma = np.sqrt(2 / sigma)
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :　重みの初期値
        """
        self.w = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return self.w
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :　バイアスの初期値
        """
        self.b = self.sigma * np.random.randn(n_nodes2)
        return self.b

In [155]:
class AdaGrad:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr, batch):
        self.lr = lr
        self.batch = batch
        # 過去の勾配の２乗和の保管用
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """

        layer.h_w += (layer.dw/ self.batch) * (layer.dw/ self.batch)
        layer.h_b += (layer.db/self.batch) * (layer.db / self.batch)
        layer.w -= self.lr * (1 / (np.sqrt(layer.h_w) + 1e-07)) * (layer.dw/ self.batch)
        layer.b -= self.lr * (1 / (np.sqrt(layer.h_b) + 1e-07)) * (layer.db/ self.batch)
        
#         layer.h_W += layer.dW * layer.dW
#         layer.h_B += layer.dB * layer.dB
#         layer.W -= self.lr * (1 / (np.sqrt(layer.h_W) + 1e-07)) * layer.dW
#         layer.B -= self.lr * (1 / (np.sqrt(layer.h_B) + 1e-07)) * layer.dB

        return layer
                                       

In [156]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=10):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

# 畳み込みクラス

In [157]:
class Conv1d:
    """
    チャネル数を限定しない１次元畳み込み層のクラス
    """
    def __init__(self, FN, FC, FS, sigma, optimizer):
        self.filter_size = FS
        self.optimizer = optimizer
        #Heの場合のシグマ
        self.sigma = np.sqrt(2 / sigma)
        # 初期化
        self.w = self.sigma * np.random.randn(FN, FC, FS)
        self.b = np.zeros((FN, 1))
        #更新用
        self.h_w = np.zeros((FN, FC, FS))
        self.h_b = ((FN, 1))
        
    def forward(self, x):
        """
        パラメータ
        ---------------------
        x : 入力値

        return
        ---------------------
        a　：　出力
        """
        self.x = x
        #出力チャネル、入力チャネル、filter数
        FN, FC,FS  = self.w.shape
        N, C, W = self.x.shape

        # 出力するaの空配列を作成
        self.n_out = np.zeros((N, FN, (W - (FS - 1)))) # (出力チャンネル数、特徴量数)
    
        # aを計算して出力
        for n in range(N):
            for i in range(FN):
                index = np.empty((0, FS), int)
                for j in range(W - (FS - 1)):
                    index_to_add = np.arange(j, j+FS)
                    index = np.append(index, [index_to_add], axis=0)
                self.n_out[n][i] = (x[n, :, index] * self.w[i]).sum(axis=1).sum(axis=1)
            self.n_out[n] += self.b
        return self.n_out  
    
    def backward(self, loss):
        """
        パラメータ
        ---------------------
        delta_a : 入力値

        return
        ---------------------
        z　：　出力
        """
        #出力チャネル、入力チャネル、filter数
        FN, FC,FS  = self.w.shape
        N, C, W = self.x.shape
        LC, LS, _ = loss.shape 
        n_out_size = self.n_out.shape[1]
        # wの勾配
#         self.dw = np.zeros([FN, FC, FS])
#         for c in range(C):
#             for fn in range(FN):
#                 for fc in range(FC):
#                     for fs in range(FS):
#                         for j in range(W-(FS-1)):
#                             self.dw[fn, fc, fs] += loss[c, fn, j]* self.x[fn, fc, j + fs]
        self.dw = np.zeros(self.w.shape)
        for n in range(N):
            for i in range(FS):
                for j in range(W - (FS - 1)):
                    self.dw[:, :, i] += loss[n, :, j][:, np.newaxis] * self.x[n, :, i+j][np.newaxis, :]

        # bの勾配計算
        self.db = loss.sum(axis=1).reshape(-1, 1)

        #デルタxの計算
        #デルタxの計算
        dx = np.zeros([C, W])
        b_1 = np.zeros([C, W])
        b_2 = np.zeros(W)
        for l in range(LC):
            for k in range(C):
                for j in range(W):
                    back_filter = 0
                    for s in range(FS):
                        if j-s < 0 or j-s > n_out_size - 1:
                            back_filter += 0 * self.w[l][k][s]
                        else:
                            back_filter += loss[l][j - s] * self.w[l][k][s]
                    b_2[j] = back_filter
                b_1[k] = b_2
            dx += b_1
            self = self.update_based(self)
            return dx
    
    def update_based(self, layer):
        layer.w -= 0.001 * layer.dw
        layer.b -= 0.001 * layer.db
        return layer
#         # 更新
#         self = self.optimizer.update(self)
#         return dZ

In [158]:
class ScratchDeepNeuralNetrowkClassifier():
    
    def __init__(self, lr=0.01,sigma=0.1, n_iter=1, batch=3, FN=3, C=1, FS=784, vervose=False):
        self.iter = n_iter
        self.lr = lr
        self.sigma = sigma
        self.batch = batch
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        self.FN = FN
        self.C = C
        self.FS = FS
        
    
    def fit(self, X, y,  X_val=None, y_val=None):
        self.n_features = X.shape[1]
        self.n_samples = X.shape[0]
        self.n_nodes1 = 400
        self.n_nodes2 =  200
        self.n_output = 10
    
        
        #更新方法の定義
        self.optimizer = AdaGrad(self.lr, self.batch)
        #　畳み込み層の定義
        self.Conv1d = Conv1d(self.FN, self.C, self.FS, self.sigma, self.optimizer)
        self.conv_activation = Relu()
        # 全結合層の定義
        self.FC3 = FC(self.FN, self.n_output, HeInitializer(self.FN), self.optimizer)
        self.activation3 = Softmax()
        
        # 学習開始
        for i in range(self.iter):
            # １エポックのloss記録保管
            history = np.zeros(X.shape[0] // self.batch)
            val_history = np.zeros(X_val.shape[0] // self.batch)
            #  バッチサイズのカウント
            count = 0
            #　ミニバッチの取り出しコード
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch)
            for mini_X_train, mini_y_train in get_mini_batch:
                # forward
                A0 = self.Conv1d.forward(mini_X_train)
                Z0 = self.conv_activation.forward(A0)
                A1 = self.FC3.forward(Z0)
                Z1 = self.activation3.forward(A1)

                # backward
                dA1, history[count] = self.activation3.backward(Z1, mini_y_train)
                dZ1 = self.FC3.backward(dA1)
                dA0 = self.conv_activation.backward(dZ1)
                dZ0 = self.Conv1d.backward(dA0)
                count += 1
                
            A0 = self.Conv1d.forward(X)
            Z0 = self.conv_activation.forward(A0)
            A1 = self.FC3.forward(Z0)
            Z1= self.activation3.forward(A1)
            
            _, history= self.activation3.backward(Z1, y)
            self.loss[i] = history
            
            # X_valあった場合の処理
            if np.any(X_val): 
                A0 = self.Conv1d.forward(X_val)
                Z0 = self.conv_activation.forward(A0)
                A1 = self.FC3.forward(Z0)
                Z1 = self.activation3.forward(A1)
                # 学習記録
                _, val_history = self.activation3.backward(Z1, y_val)
                self.val_loss[i] = val_history
            
            
    # 推定
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        A0 = self.Conv1d.forward(X)
        Z0 = self.conv_activation(A0)
        A1 = self.FC3.forward(Z0)
        Z1 = self.activation3.forward(A1)
        return np.argmax(Z1, axis= 1)

In [159]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [160]:
# One-Hot-encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# データの前処理
X_train = X_train.reshape(-1, 1, 784)
X_test = X_test.reshape(-1, 1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.fit_transform(y_test[:,  np.newaxis])
#訓練データから更に検証データを生成
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [161]:
from sklearn.metrics import accuracy_score
SDNNC = ScratchDeepNeuralNetrowkClassifier()

In [162]:
SDNNC.fit(X_train, y_train, X_val, y_val)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply


KeyboardInterrupt: 

In [ ]:
y_pred = SDNNC.predict(X_test)

In [164]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_test_one_hot))

NameError: name 'y_pred' is not defined

In [ ]:
処理が重く回しきれなかったので中断しました。